In [ ]:
import openai
from qdrant_client import QdrantClient

In [ ]:
# Define the embedding model to use
embedding_model = "text-embedding-3-small"
# Define the collection name in qdrant database
collection_name = "Amazon-items-collection-00"
# Define the OpenaI language model to use
model = "gpt-5-nano"

In [ ]:
# Embedding function; same from previous notebook
def get_embedding(text, model=embedding_model):
    response = openai.embeddings.create(
        input=text,
        model=model,
    )
    return response.data[0].embedding

In [ ]:
# Create the Qdrant client (ensure first to have Qdrant server running locally via docker compose;
# Then define the function for data retrieval from Qdrant based on a query text
# top K similar items will be retrieved
qdrant_client = QdrantClient(url="http://localhost:6333")
def retrieve_data(query, qdrant_client, k=5):

    query_embedding = get_embedding(query)

    results = qdrant_client.query_points(
        collection_name=collection_name,
        query=query_embedding,
        limit=k,
    )

    retrieved_context_ids = []
    retrieved_context = []
    similarity_scores = []
    retrieved_context_ratings = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        retrieved_context_ratings.append(result.payload["average_rating"])
        similarity_scores.append(result.score)

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "retrieved_context_ratings": retrieved_context_ratings,
        "similarity_scores": similarity_scores,
    }

In [ ]:
# Finally test the retrieval function
retrieved_context = retrieve_data(
    "What kind of earphones can I get?", qdrant_client, k=10
)
# And print the context retrieved
retrieved_context

In [ ]:
# Now format the context before using it inside the prompt
def process_context(context):

    formatted_context = ""

    for id, chunk, rating in zip(
        context["retrieved_context_ids"],
        context["retrieved_context"],
        context["retrieved_context_ratings"],
    ):
        formatted_context += f"- ID: {id}, rating: {rating}, description: {chunk}\n"

    return formatted_context

preprocessed_context = process_context(retrieved_context)
print(preprocessed_context)

In [ ]:
# Create the prompt with the enriched context
def build_prompt(preprocessed_context, question):

    prompt = f"""
You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.

Context:
{preprocessed_context}

Question:
{question}
"""

    return prompt

In [ ]:
# Build the prompt starting from the original user query
prompt = build_prompt(preprocessed_context, "What kind of earphones can I get?")
print(prompt)

In [ ]:
# Generate the answer with the enriched prompt
def generate_answer(prompt):

    response = openai.chat.completions.create(
        model=model,
        messages=[{"role": "system", "content": prompt}],
        reasoning_effort="minimal",
    )

    return response.choices[0].message.content

print(generate_answer(prompt))

In [ ]:
# Create a naive RAG pipeline that assembles all the previous steps
def rag_pipeline(question, top_k=5):

    qdrant_client = QdrantClient(url="http://localhost:6333")

    retrieved_context = retrieve_data(question, qdrant_client, top_k)
    preprocessed_context = process_context(retrieved_context)
    prompt = build_prompt(preprocessed_context, question)
    answer = generate_answer(prompt)

    return answer

# Finally test the retrieval function
print(rag_pipeline("What kind of earphones can I get with ratings above 4.5?"))